# CityBikes

Send a request to CityBikes for the city of your choice. 

In [37]:
import requests
import pandas as pd

In [38]:
id = "rva-bikes"
url = "http://api.citybik.es/v2/networks/" + id
result = requests.get(url)
data=result.json()

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [39]:
stationjson=data['network']['stations']
stationjson

[{'id': '12b6d59eb26b7368c6a4777c7a37a441',
  'name': 'East End Library',
  'latitude': 37.539778925345864,
  'longitude': -77.41146002560403,
  'timestamp': '2024-10-16T03:51:55.977958Z',
  'free_bikes': 0,
  'empty_slots': 6,
  'extra': {'uid': 'f24cafdd-ef3c-48c4-b05c-3ed632613880',
   'description': 'East End Library',
   'address': '1200 N 25th St, Richmond, VA 23223',
   'online': False,
   'last_update': 1724080159420,
   'photo': 'https://api.rvabikes.com/cxf/fm/files/00000000-0000-0000-0000-000000000000'}},
 {'id': '2b1b59f43c955e8b45e9218c5ff8b54c',
  'name': 'Sydney Park',
  'latitude': 37.54696765790552,
  'longitude': -77.45659857991984,
  'timestamp': '2024-10-16T03:51:55.978055Z',
  'free_bikes': 0,
  'empty_slots': 16,
  'extra': {'uid': 'f1708f70-8c41-4e8b-9670-f35d655b8c44',
   'description': 'Triangle park',
   'address': '26 N. Morris Street',
   'online': False,
   'last_update': 1724282379065,
   'photo': 'https://api.rvabikes.com/cxf/fm/files/00000000-0000-0000-0

Put your parsed results into a DataFrame.

In [35]:
df=pd.DataFrame(stationjson,columns=['name','latitude','longitude','free_bikes','empty_slots'])
df['total_bike_cap']=df['empty_slots']+df['free_bikes']